In [1]:
import pandas as pd

class Stock(object):
    def __init__(self, symbol, data):
        self.symbol = symbol
        self.data = data
        self.data.Date = pd.to_datetime(self.data.Date, format="%Y-%m-%d")
        self.compute_stats()

    def compute_stats(self, price_header='Close'):
        self.open_diffs = self.data[price_header].pct_change()[1:]

        self.avg = self.open_diffs.mean()
        self.var = self.open_diffs.var()
        self.std = self.open_diffs.std()

    def get_sym(self):
        return self.symbol

In [5]:
# from yfapi import YahooFinanceAPI, Interval
# from stock import Stock
from pandas_datareader import data

import datetime
import matplotlib.pyplot as plt
import pandas as pd
from numpy import random
import numpy as np
from dateutil.relativedelta import relativedelta
from numpy.fft import fft

class GBMMonteCarlo(object):
    def __init__(self):
        pass

    def run(self, ticker_list, simulations, final_day, data_months=1, plot_results=True):
        self.simulations = simulations
        self.final_day = datetime.datetime.strptime(final_day, "%Y-%m-%d")
        self.data_months = data_months
        self.current = datetime.datetime.today()
        self.T = (self.final_day - self.current).days + 2

        if self.T <= 0:
            print("Final day is on or after the current day.")
            exit()

        self.removed_dates = []
        for d in range(self.T):
            date = (self.current + relativedelta(days=d))
            if date.weekday() > 4: #or (date in holidays):
                self.removed_dates.append(date)
                self.T -= 1

#         dh = YahooFinanceAPI(interval=Interval.DAILY)
        symbols = ticker_list
        portfolio = {}

        past = self.current - relativedelta(days=int(self.data_months*30))
#         portfolio = dh.get_data_for_tickers(symbols, past, self.current)
        portfolio = data.DataReader(symbols,'yahoo', past, self.current)
        portfolio = portfolio['Adj Close']
        for sym in symbols:
            portfolio[sym] = Stock(sym, portfolio[sym])

        paths = {}
        for sym in symbols:
            stock = portfolio[sym]
            s_0 = stock.get_last_price()
            sig = stock.get_daily_return_std()
            sig2 = sig**2
            mu = stock.get_daily_return_avg()

            # random daily shocks to the stock price (price fluctuations)
            shocks = {sim: random.normal(0, 1, self.T) for sim in range(self.simulations)}
            # cumulative sum of all previous shocks to the stock price (price path)
            brownian_motion = {sim: shocks[sim].cumsum() for sim in range(self.simulations)}
            paths[stock.get_sym()] = [[s_0] for _ in range(self.simulations)]

            for i in range(self.simulations):
                paths[stock.get_sym()][i].extend(\
                    [s_0 * np.exp(((mu - 0.5*sig2)*j) + sig*brownian_motion[i][j-1]) for j in range(1, self.T+1)]
                )

        if plot_results:
            self.plot(paths, portfolio, symbols)
        return paths

    def plot(self, paths, portfolio, symbols):
        max_idx_dict = {}
        min_idx_dict = {}

        for sym in symbols:
            max_amt = -1000000000
            min_amt = 1000000000
            last_idx = len(paths[sym][0]) - 1

            for i in range(self.simulations):
                price = paths[sym][i][last_idx]
                if price < min_amt: # lowest final price
                    min_amt = price
                    min_idx_dict[sym] = i
                if price > max_amt: # highest final price
                    max_amt = price
                    max_idx_dict[sym] = i

        avg_price_dict = {}
        for sym in symbols:
            avg_price_dict[sym] = [portfolio[sym].data.Close[len(portfolio[sym].data) - 1]]
            for i in range(self.T):
                avg = 0.0
                for j in range(self.simulations):
                    avg += paths[sym][j][i]
                avg_price_dict[sym].append(float(avg)/float(self.simulations))


        # plot the simulation data
        fig, (ax1, ax2) = plt.subplots(2, 2)

        dates = portfolio[sym].data.Date.tolist()
        new_dates = [dates[-1]]
        new_dates.extend(pd.date_range(start=self.current, end=(self.final_day + relativedelta(days=1))).tolist())
        new_dates = [date for date in new_dates if date not in self.removed_dates]

        fig.suptitle("Simulation Results")
        colors = [np.random.rand(3,) for _ in range(len(symbols))]
        count = 0
        for sym in symbols:
            open = portfolio[sym].data.Close.tolist()

            ax1[0].plot(dates, open, color=colors[count], label=sym.upper())
            for i in range(self.simulations):
                ax1[0].plot(new_dates, paths[sym][i], color=np.random.rand(3, ))

            ax1[1].plot(dates, open, color=colors[count])
            ax1[1].plot(new_dates, avg_price_dict[sym], color="orange", label="Average")
            ax1[1].plot(new_dates, paths[sym][max_idx_dict[sym]], color="green", label="Best")
            ax1[1].plot(new_dates, paths[sym][min_idx_dict[sym]], color="red", label="Worst")

            x = [range(self.T+1) for _ in range(self.simulations)]
            ax2[0].scatter(x, paths[sym], color=colors[count], label=sym.upper())

            ax2[1].plot(portfolio[sym].data.Date[1:], portfolio[sym].open_diffs, \
                     color=colors[count], label=sym.upper())

            if count == 0:
                ax1[1].legend()
                ax1[1].set_title("Predicitons")

            if count == (len(symbols) -1):
                ax1[0].legend()
                ax1[0].set_title("Simulations")
                ax2[0].legend()
                ax2[0].set_title("Price Distribution")
                ax2[1].set_title("Daily Changes")
                ax2[1].legend()

                ax1[0].tick_params(axis='x', labelrotation=30)
                ax1[1].tick_params(axis='x', labelrotation=30)
                ax2[1].tick_params(axis='x', labelrotation=30)

            count += 1
        plt.show()

if __name__ == '__main__':
    gbmmc = GBMMonteCarlo()
    gbmmc.run(["mlr", "gntx"], 50, '2020-10-29', data_months=2)

Final day is on or after the current day.


AttributeError: 'Series' object has no attribute 'Date'